## **LLM Fine tuning**

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
dataset = load_dataset("kaitchup/ultrachat-100k-flattened")

In [ ]:
compute_dtype = getattr(torch, "bfloat16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
    cache_dir=f"./models",
    local_files_only=True
)
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias=None,
    task_type="CASUAL_LM",
    target_modules=["k_proj", "q_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]
)
training_arguments = TrainingArguments(
    output_dir="./model/mistral7b_results",
    evaluation_strategy="steps",
    do_eval=True,
    optim="paged_adamw_8bit",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    gradient_accumulation_steps=2,
    log_level="debug",
    logging_steps=100,
    learning_rate=1e-4,
    eval_steps=25,
    max_steps=100,
    save_steps=25,
    warmup_steps=25,
    lr_scheduler_type="linear"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments
)

trainer.train()

In [ ]:
from transformers import GenerationConfig
model.config.use_cache = True
model = PeftModel.from_pretrained(model, "./model/mistral7b_results/checkpoint-100/")

def generate(instruction):
    prompt = "### Human" + instruction + "### Assistant:"
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(pad_token_id=tokenizer.pad_token_id, temperature=1.0, top_p=1.0, top_k=50, num_beams=1),
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(output.split("### Assistant: ")[1].strip())

generate("Tell me about gravitation.")